<a href="https://colab.research.google.com/github/alexttrt/Team5_adopt_linebot_project/blob/main/Team5_linebot_v1_ipnyb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install line-bot-sdk flask_ngrok
!pip install pyngrok
!ngrok authtoken 2SIRRD9b7E9IGNhqxvd4HNlyuP8_2wekze8q4VQJHXKKb2HbS

In [ ]:
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok

from flask import Flask
from flask import Flask, request, abort, jsonify

from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import FlexSendMessage, BubbleContainer, ImageComponent
from linebot.models import (MessageEvent, TextMessage, TextSendMessage, ImageSendMessage, TemplateSendMessage, StickerSendMessage, ImageMessage, PostbackEvent)
from linebot.models import MessageAction, TemplateSendMessage, ImageCarouselTemplate, ImageCarouselColumn
from linebot.models.events import (FollowEvent)
from linebot.models import PostbackAction, URIAction, MessageAction, TemplateSendMessage, ButtonsTemplate

from linebot.models.template import(ButtonsTemplate)
from linebot.models import *

import urllib.request
import json
import torch
from torch import nn
import torchvision
from torchvision import models, transforms
from torchsummary import summary
from PIL import Image
import io

In [ ]:
# 連結雲端硬碟
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

with open("env.json") as f :
  env = json.load(f)
# 生成實體物件
line_bot_api = LineBotApi(env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])
label = env['YOUR_LABELS']
with open(label, "r") as f:
  labels = (f.read().split())
print(labels)

['黑貓', '白貓', '橘貓', '黑白貓', '三花貓', '虎斑貓', '玳瑁貓']


<ipython-input-40-9cdddeb85494>:8: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(env['YOUR_CHANNEL_ACCESS_TOKEN'])
<ipython-input-40-9cdddeb85494>:9: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])


In [ ]:
print(env['YOUR_CHANNEL_ACCESS_TOKEN'])

U+3/UmH7jWMtE58/8CHaJbBujNpxFBeCZsKKZAVER94q9w3XwVGOa7bNB2/2d3Hvt4k9Qm0elCIlt35ysdF4o975/urYzP2NINVyMpHshFmHXhl62GOAza12ljFU/VKELfpyP4GgDgZwSf6vEo6UHgdB04t89/1O/w1cDnyilFU=


In [ ]:
# 辨識圖片預處理
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),  # 調整圖片大小
    transforms.ToTensor(),  # 將圖片轉為 Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 正規化圖片
])

In [ ]:
# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
# 載入Follow事件
# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):

    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)

     # 將用戶資訊存在檔案內
    with open("/content/drive/MyDrive/users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\n')

    # 回覆歡迎文字消息
    line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage(
        '''您好~
          歡迎加入智慧寵物喵!
          這裡提供各式便捷的服務
          快點來體驗看看吧
          請輸入@start
          '''),
         StickerSendMessage(
         package_id="1070",
         sticker_id="17839"
         ),
         quick_reply]

          )


In [ ]:
quick_reply= TextSendMessage(
                text='快來領養毛小孩吧?',
                quick_reply=QuickReply(
                    items=[
                        QuickReplyButton(
                            action=PostbackAction(label="主選單", data="postback1")
                        ),
                        QuickReplyButton(
                            action=MessageAction(label="繼續下一步", text="繼續流程")
                        ),

                        QuickReplyButton(
                            action=CameraRollAction(label="辨識更多")
                        ),
                        QuickReplyButton(
                            action=CameraAction(label="開啟相機吧")
                        ),
                        QuickReplyButton(
                            action=DatetimePickerAction(label="時間選單",
                                                        data ="date_postback",
                                                        mode ="date")
                        )
                    ]))

In [ ]:
def classify(image):
    # image_path = image
    # image = Image.open(image_path)
    # 預處理圖片
    input_tensor = preprocess(image)
    input_batch = input_tensor.unsqueeze(0)
    # 將圖片放入模型預測
    model = torch.load('/content/resnet50_model.pth', map_location=torch.device('cpu'))
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    input_batch = input_batch.to(device)
    with torch.no_grad():
      output = model(input_batch)
    predicted_idx = torch.max(output, 1)
    predicted_class = predicted_idx[1].item()
    predicted_label = labels[predicted_class]
    return predicted_label

In [ ]:
#收到用戶回應時的處理方式
#收到圖片訊息處理
@handler.add(MessageEvent, message=ImageMessage)
def handle_img_message(event):
    message_id = event.message.id
    message_content = line_bot_api.get_message_content(message_id)


    b = b''
    for chunk in message_content.iter_content():
        b += chunk
    img = Image.open(io.BytesIO(b))
    r = classify(img)

    line_bot_api.reply_message(
      event.reply_token,
      TextSendMessage(text=f"辨識結果為:{r}"))




#用戶點擊button後，觸發postback event，對其回傳做相對應處理
#處理模板按鈕的訊息

@handler.add(PostbackEvent)
def handle_post_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    if (event.postback.data.find('action1')== 0):
        line_bot_api.reply_message(
            event.reply_token,
                breakingnews_template
            )
    elif (event.postback.data.find('action2') == 0):
        line_bot_api.reply_message(
            event.reply_token,
                adoptprocess_template
            )
    elif (event.postback.data.find('action3') == 0):
        line_bot_api.reply_message(
            event.reply_token,
                petsdaily_template
            )

    elif (event.postback.data.find('action4') == 0):
        line_bot_api.reply_message(
            event.reply_token,
                outdoor_template
            )

    elif (event.postback.data.find('領養須知') == 0):
        line_bot_api.reply_message(
            event.reply_token,
                [section1, section2, section3, confirm])

    elif (event.postback.data.find('領養程序') == 0):
        line_bot_api.reply_message(
            event.reply_token,
                [TextSendMessage(text="感謝您的愛心 讓我帶領你走一趟流程吧!"),
                 confirm2])
    elif (event.postback.data.find('postback1') == 0):
        line_bot_api.reply_message(
            event.reply_token,
                main_template_message)

    else:
        pass


# 收到用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):

    if(event.message.text.find('@')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict3.get(event.message.text)
        )

    elif(event.message.text in north ):
        line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage(text="下面是您居住地附近的收容所"),
        northdistrict,
         fill_doc,
         quick_reply])

    elif(event.message.text == "我已經完全了解!" ):
        line_bot_api.reply_message(
        event.reply_token,
        [notice1,notice2,confirm3])
    elif(event.message.text == "繼續流程"):
        line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage(text="讓我帶您了解附近的收容所"),
         TextSendMessage(text="請輸入居住縣市")])
    elif(event.message.text == "我先看看有哪些"):
        line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage(text="讓我帶您了解附近的收容所"),
         TextSendMessage(text="請輸入居住縣市")])
    elif(event.message.text == "重新上傳" ):
        line_bot_api.reply_message(
        event.reply_token,
        confirm2)
    elif(event.message.text == "回主選單" ):
        line_bot_api.reply_message(
        event.reply_token,
        main_template_message)
    elif(event.message.text == "可領養的喵喵" ):
        line_bot_api.reply_message(
        event.reply_token,
        unadopted_cat)
    elif(event.message.text == "我要通報!" ):
        line_bot_api.reply_message(
        event.reply_token,
        missing_confirm)



    else:
      pass

    if(event.message.text.find('#筆記')!= -1):

      line_bot_api.reply_message(
            event.reply_token,
                TextSendMessage(text="已將您的筆記內容記錄到筆記本 若要查看可使用#查看筆記")
            )
      with open("心得筆記.txt","a")as file:
            file.write(event.message.text+"\n")
            file.write("======"*3+"\n")

    if(event.message.text.find('#查看筆記')!= -1):
      with open("心得筆記.txt","r",encoding="utf-8") as file:
         text1=file.read()
         line_bot_api.reply_message(
            event.reply_token,
                [TextSendMessage(text="以下是您的記事本內容"),TextSendMessage(text=text1)]
            )
    if(event.message.text.find('!回報')!= -1):
      line_bot_api.reply_message(
            event.reply_token,
                TextSendMessage(text="已記錄您的回報內容")
            )
      with open("回報記錄.txt","a")as file:
            file.write(event.message.text+"\n")
            file.write("======"*3+"\n")


In [ ]:
breakingnews_dict={}
adoptprocess_dict={}
petsdaily_dict={}
outdoor_dict={}

#文字訊息
#領養貓咪前的準備
section1=TextSendMessage("決定要領養幼貓還是成貓\n\n"+'''認養貓咪前最重要的一個決定就是要選擇成貓還是幼貓了！
雖然大多數人一開始可能會因為小貓可愛的模樣而想認養幼貓。
但其實認養幼貓不僅需要在食物、保暖、醫療上花更多的心力和時間照顧，
更因為小貓個性還不穩定容易有咬人、抓人、破壞家具等行為而必須額外訓練！
因此若是沒有經驗、時間比較少又沒有特別想要小貓的話
建議還是領養成貓比較適合！
''')

section2=TextSendMessage("確定自己有時間、地點和金錢照顧貓咪\n\n"+'''貓咪的壽命可以長達 10 – 14 歲，
因此在領養前要先確定自己可以在那麼長的時間內
都可以負擔起照顧他們的責任。現在的工作有時間陪伴他們嗎？
之後有可能換成比較忙的工作，甚至出國嗎？租屋的話可否養寵物？
之後搬家的話呢？另外貓咪的飼料、貓砂、醫療等一個月需要花上 3000 – 5000 塊，
現在的經濟情況是否允許？家人是否同意等等都是需要仔細思考的問題！
''')

section3=TextSendMessage("多了解關於照顧貓咪文章和內容\n\n"+'''在領養貓咪前，建議最好先了解一些例如：
貓咪該吃什麼？什麼不能吃？貓砂怎麼換？貓咪需要洗澡嗎？
等等照顧貓咪的基本知識。不僅可以讓你做好心理準備，
帶回貓咪後也比較不會手忙腳亂。
''')

#貓咪認養時的注意事項
notice1=TextSendMessage("詢問貓咪的身體狀況\n\n"+'''認養貓咪前一定要先確認貓咪的健康狀況！
詢問送養者貓咪的年齡、性別、是否結紮、有無先天或後天疾病、疫苗接種狀況等等，
一般情形下送養者都會確實告知。建議沒有經驗的話盡量不要養本身就有疾病的貓，
會需要非常多額外的心力。若真的要養病貓的話也建議領養前多學習相關的照護知識！
''')

notice2=TextSendMessage("事前熟悉貓咪的個性\n\n"+'''跟人一樣，每隻貓咪也有自己的個性。有些就是比較親人愛撒嬌，
有些則是看到你就一溜煙跑的不見貓影。貓咪的個性是不太會改變的，
所以如果想要跟自己的貓咪有點親密時光，不想養得很ㄘㄟˋ心的話，
建議可以的話在領養前實際跟貓咪玩玩，相處一段時間 ，確定貓咪的親人程度再進行領養！
''')

In [ ]:
#主選單建立
main_template_message= TemplateSendMessage(
    alt_text='ButtonsTemplate',
    template=ButtonsTemplate(
        thumbnail_image_url='https://scontent-tpe1-1.xx.fbcdn.net/v/t39.30808-6/357743504_6618450718176561_8982136937443641572_n.jpg?_nc_cat=109&ccb=1-7&_nc_sid=730e14&_nc_ohc=uCaLy3sMqZIAX9KRiwn&_nc_ht=scontent-tpe1-1.xx&oh=00_AfD_bUerrTnxZO00FzUWlBafpeEfViJWDcjtr56jux2CjQ&oe=64ADF50F',
        title='喵喵主選單',
        text='需要什麼服務呢?',
        actions=[
            PostbackAction(
                label='最新消息',
                text="想知道什麼消息都在這!",
                data='action1'
            ),
            PostbackAction(
                label='我要認養',
                text="帶我了解領養流程",
                data='action2'
            ),
            PostbackAction(
                label='寵物日常',
                text="記錄愛貓的生活",
                data='action3'
            ),
            PostbackAction(
                label='出去遛遛',
                text="想帶愛貓出去玩~",
                data='action4'
            )
        ]
    )
)
#副選單模板 1.
breakingnews_template= TemplateSendMessage(
    alt_text='ButtonsTemplate',
    template=ButtonsTemplate(
        thumbnail_image_url= "https://www.supercoloring.com/sites/default/files/styles/coloring_full/public/cif/2019/11/cat-broadcast-announcer-coloring-page.png",
        title='寵物界的大小事',
        text='將你的所見所聞變得更廣為人知吧!',
        actions=[
            PostbackAction(
                label="新增收容",
                text="可領養的喵喵",
                data='動作1'
            ),
            PostbackAction(
                label='走失協尋',
                text="我要通報!",
                data='動作2'
            ),
        ]
    )
)

#副選單模板 2.
adoptprocess_template= TemplateSendMessage(
    alt_text='ButtonsTemplate',
    template=ButtonsTemplate(
        thumbnail_image_url= "https://scontent-tpe1-1.xx.fbcdn.net/v/t39.30808-6/358391254_6618537664834533_1338829207264939757_n.jpg?_nc_cat=108&ccb=1-7&_nc_sid=730e14&_nc_ohc=WqjiXEYmhzoAX9Qql_a&_nc_ht=scontent-tpe1-1.xx&oh=00_AfCHLPZ54uZVTdULOAjTtkao-qpv3wv29bLHsJp7TpzykA&oe=64AE4A4C",
        title="尋找毛小孩",
        text="呼籲以領養代替購買喔!",
        actions=[
            PostbackAction(
                label="基本領養需知",
                text="我想了解領養前要有甚麼準備?",
                data='領養須知'
            ),
            PostbackAction(
                label="進入領養程序",
                text="我已了解領養流程 開始領養程序",
                data='領養程序'
            )
        ]
    )
)

#副選單模板 3.
petsdaily_template= TemplateSendMessage(
    alt_text='ButtonsTemplate',
    template=ButtonsTemplate(
        thumbnail_image_url= "https://scontent-tpe1-1.xx.fbcdn.net/v/t39.30808-6/359324996_6619608981394068_5502915817996258193_n.jpg?_nc_cat=103&ccb=1-7&_nc_sid=730e14&_nc_ohc=OOq56DketeAAX9SbxB_&_nc_ht=scontent-tpe1-1.xx&oh=00_AfA-CXbrzkJx49WG9BOcA69pdmGY-yFuIPe6BCQx-swCVw&oe=64AE9E07",
        title='毛小孩的生活管家',
        text='毛小孩不會講話...記得餵她吃飯',
        actions=[
            MessageAction(
                label="活動紀錄",
                text="這裡的紀錄會記在google sheet裡",

            ),
            MessageAction(
                label='飲食記錄',
                text="這裡的紀錄會記在google sheet裡",

            ),
            MessageAction(
                label="病歷紀錄",
                text="這裡的紀錄會記在google sheet裡",

            ),
            MessageAction(
                label="獸醫掛號",
                text="這裡的紀錄會記在google sheet裡",

            )

        ]
    )
)

#副選單模板 4.
outdoor_template= TemplateSendMessage(
    alt_text='ButtonsTemplate',
    template=ButtonsTemplate(
        thumbnail_image_url= "https://scontent-tpe1-1.xx.fbcdn.net/v/t39.30808-6/359351398_6618907874797512_6932741876875671605_n.jpg?_nc_cat=110&ccb=1-7&_nc_sid=730e14&_nc_ohc=kHZxjf621fkAX9oQ9gA&_nc_ht=scontent-tpe1-1.xx&oh=00_AfB3FY2Q62ITjCu9sVuOr0T9oN_NYydGjOUUaLPVhoBWsQ&oe=64AF3C9F",
        title='帶毛小孩出門曬太陽',
        text='不知道哪裡有寵物友善空間嗎?',
        actions=[
            URIAction(
                label="寵物友善餐廳",
                uri="https://24h.pchome.com.tw/prod/DEBQKJ-A9008QVKG"


            ),
            URIAction(
                label="寵物旅館",
                uri="https://24h.pchome.com.tw/prod/DEBQ5E-A900G1CO2"


            ),
            URIAction(
                label="寵物公園",
                uri="https://24h.pchome.com.tw/prod/DEBQOS-A900EVNC1"


            )
        ]
    )
)

#確認選單
confirm= TemplateSendMessage(
    alt_text='ConfirmTemplate',
    template=ConfirmTemplate(
        text='以上為領養前準備 有了解了嗎?',
        actions=[
            MessageAction(
                label='了解',
                text="我已經完全了解!"
            ),
            MessageAction(
                label='尚未',
                text="讓我再看一下"
            ),

        ]
    )
)

confirm2= TemplateSendMessage(
    alt_text='ConfirmTemplate',
    template=ConfirmTemplate(
        text='喜歡哪種毛小孩呢?',
        actions=[
            URIAction(
                label='圖片辨識',
                uri='https://line.me/R/nv/cameraRoll/single'),
            MessageAction(
                label='去逛逛',
                text="我先看看有哪些",
            ),

        ]
    )
)
confirm3= TemplateSendMessage(
    alt_text='ConfirmTemplate',
    template=ConfirmTemplate(
        text='以上是見到喵喵本人時要確認的事情 準備好了嗎?',
        actions=[
            PostbackAction(
                label="準備好了",
                text="我已全部了解! 帶我開始領養程序吧",
                data='領養程序'
            ),
            MessageAction(
                label='尚未',
                text="讓我再看一下",

            ),

        ]
    )
)
notconfirm= TemplateSendMessage(
    alt_text='ButtonsTemplate',
    template=ButtonsTemplate(
        thumbnail_image_url= "https://blog.tripbaa.com/wp-content/uploads/2019/08/plan_201908091808307153_b-1024x575.jpg",
        title='了解了嗎?',
        text='提供兩種方式解決',
        actions=[
            URIAction(
                label='野外實用小技能',
                uri='https://www.prooutdoor.com.tw/blogs/mountain-secret/or-%E5%B1%B1%E6%9E%97%E5%AF%A6%E9%A9%97%E5%AE%A4-%E5%B1%B1%E6%9E%97%E7%94%9F%E5%AD%98%E5%AF%86%E6%8A%80-%E5%A6%82%E4%BD%95%E5%9C%A8%E9%87%8E%E5%9C%B0%E5%8F%96%E6%9D%90%E7%94%9F%E7%81%AB',

            ),
            MessageAction(
                label='好用登山小物',
                text="錢能解決的都不是問題",

            ),

        ]
    )
)


In [ ]:
#走失通報
missing_confirm= TemplateSendMessage(
    alt_text='ConfirmTemplate',
    template=ConfirmTemplate(
        text='你要通報哪一種呢',
        actions=[
            MessageAction(
                label='協尋通報',
                text="我家毛小孩走失了 麻煩大家幫我找找!",
            ),
            MessageAction(
                label='路上找到',
                text="我好像看到走失的貓貓",

            ),

        ]
    )
)

In [ ]:
#居住區域劃分
north=["台北市","新北市","基隆市","桃園市","新竹市","新竹縣","宜蘭縣"]
mid=["苗栗縣","台中市","彰化縣","南投縣","雲林縣"]
south=["嘉義市","嘉義縣","台南市","高雄市","屏東縣","澎湖縣"]

#收容所區域
#圖片輪播範本
#台灣各區收容所
northdistrict=TemplateSendMessage(
    alt_text='ImageCarouselTemplate',
    template=ImageCarouselTemplate(
        columns=[
            ImageCarouselColumn(
                image_url='https://scontent-tpe1-1.xx.fbcdn.net/v/t39.30808-6/326650117_863447528222413_3496208209142595679_n.png?_nc_cat=102&ccb=1-7&_nc_sid=09cbfe&_nc_ohc=El5Mz-zggQoAX_ry1gS&_nc_ht=scontent-tpe1-1.xx&oh=00_AfDtHpmSq38bAUjI4hIWc0SVGYXsH9nZByoMGWyVcqHR4Q&oe=64AEDB08',
                action=URIAction(
                label="臺北市動物之家",
                uri='https://www.facebook.com/tcapo.tas/'
                )
            ),
            ImageCarouselColumn(
                image_url='https://www.ahiqo.ntpc.gov.tw/upload/cht/article/72025406353f19414d1e46361db0417b.jpg',
                action=URIAction(
                    label='新北市政府動物保護防疫處',
                    uri='https://www.ahiqo.ntpc.gov.tw/cht/index.php?'
                )
            ),
            ImageCarouselColumn(
                image_url='https://taw.tycg.gov.tw/images/TOP-1170.jpg',
                action=URIAction(
                    label='桃園市動物保護教育園區',
                    uri="https://taw.tycg.gov.tw/X00_index.aspx"
                )
            ),
            ImageCarouselColumn(
                image_url='https://scontent-tpe1-1.xx.fbcdn.net/v/t39.30808-6/357736134_6619687564719543_2303349824690039417_n.jpg?_nc_cat=110&ccb=1-7&_nc_sid=730e14&_nc_ohc=bA5xejT3dPYAX-1NyA8&_nc_ht=scontent-tpe1-1.xx&oh=00_AfAZPihpN06QDvoqtBtXmHSUyZdHpG1kpbYy9sjX_I6g7Q&oe=64AE35CB',
                action=URIAction(
                    label='宜蘭縣流浪動物中途之家',
                    uri="https://animal.e-land.gov.tw/Default.aspx"
                )
            ),
            ImageCarouselColumn(
                image_url='https://scontent-tpe1-1.xx.fbcdn.net/v/t39.30808-6/357732749_6619692518052381_422083509601730492_n.jpg?_nc_cat=100&ccb=1-7&_nc_sid=730e14&_nc_ohc=f5SbWUZIWxUAX8f0MW8&_nc_ht=scontent-tpe1-1.xx&oh=00_AfBehFRYZVmFjQWRcN0hSH168fZzgA9xOcp5rpxtycO7dw&oe=64AE1B43',
                action=URIAction(
                    label='新竹市動物保護教育園區',
                    uri="https://puppy.hccg.gov.tw/ch/home.jsp?id=6&parentpath=0"
                )
            ),
            ImageCarouselColumn(
                image_url='https://scontent-tpe1-1.xx.fbcdn.net/v/t39.30808-6/358966186_6619692448052388_8845866107030886513_n.jpg?_nc_cat=103&ccb=1-7&_nc_sid=730e14&_nc_ohc=wrUHEPo5QhwAX-sYFoR&_nc_oc=AQkrbzcDYIZCG_3CbpZIUwlZYtAgIfAVM0SVnvktWiXGsErWU4fEv9WPVGsC3qtvpkM&_nc_ht=scontent-tpe1-1.xx&oh=00_AfBIlQ_HJ1MimPmjzA76so3dGkw_OgcfMyZO3LSS1ZQGFQ&oe=64AE29FC',
                action=URIAction(
                    label='基隆市寵物銀行',
                    uri="https://pet234.com/shop-a156f2e9-06db-4310-9b13-d1b4a2f42518.html"
                )
            )
        ]
    )
)

middistrict=TemplateSendMessage(
    alt_text='ImageCarouselTemplate',
    template=ImageCarouselTemplate(
        columns=[
            ImageCarouselColumn(
                image_url='https://www.animal.taichung.gov.tw/1521490/Lpsimplelist',
                action=URIAction(
                    label='臺中市動物之家南屯園區',
                    uri="https://www.animal.taichung.gov.tw/1521490/Lpsimplelist"
                )
            ),
            ImageCarouselColumn(
                image_url='https://www.ahiqo.ntpc.gov.tw/cht/index.php?',
                action=URIAction(
                    label='苗栗縣生態保育教育中心(動物收容所)',
                    uri="https://www.ahiqo.ntpc.gov.tw/cht/index.php?"
                )
            ),
            ImageCarouselColumn(
                image_url='https://www.chcgadcc.gov.tw/',
                action=URIAction(
                    label='彰化縣流浪狗中途之家',
                    uri="https://www.chcgadcc.gov.tw/"
                )
            ),
            ImageCarouselColumn(
                image_url='http://adcc.nantou.gov.tw/introduce/leading.asp?id=%7BAAF1647F-051F-47C0-BCFC-03E9C497E4D2%7D',
                action=URIAction(
                    label='南投縣公立動物收容所',
                    uri="'http://adcc.nantou.gov.tw/introduce/leading.asp?id=%7BAAF1647F-051F-47C0-BCFC-03E9C497E4D2%7D'"
                )
            )
        ]
    )
)

southdistrict=TemplateSendMessage(
    alt_text='ImageCarouselTemplate',
    template=ImageCarouselTemplate(
        columns=[
            ImageCarouselColumn(
                image_url='https://ahipo.tainan.gov.tw/News.aspx?n=5388&sms=10190',
                action=MessageAction(
                    label='臺南市動物之家灣裡站',
                    text="功能建立中....QQ"
                )
            ),
            ImageCarouselColumn(
                image_url='https://ahipo.tainan.gov.tw/News.aspx?n=5389&sms=10191',
                action=MessageAction(
                    label='臺南市動物之家善化站',
                    text="功能建立中....QQ"
                )
            ),
            ImageCarouselColumn(
                image_url='https://livestock.yunlin.gov.tw/News.aspx?n=910&sms=9439',
                action=MessageAction(
                    label='雲林縣流浪動物收容所',
                    text="功能建立中....QQ"
                )
            ),
            ImageCarouselColumn(
                image_url='https://www.facebook.com/pages/category/Interest/%E9%9D%9E%E5%AE%98%E6%96%B9%E5%98%89%E7%BE%A9%E7%B8%A3%E6%B0%91%E9%9B%84%E6%94%B6%E5%AE%B9%E6%89%80%E6%B5%81%E6%B5%AA%E7%8A%AC%E4%B8%AD%E9%80%94%E4%B9%8B%E5%AE%B6-1069116556450723/',
                action=MessageAction(
                    label='嘉義縣流浪犬中途之家',
                    text="功能建立中....QQ"
                )
            ),
            ImageCarouselColumn(
                image_url='https://www.facebook.com/adoptmeloveme/',
                action=MessageAction(
                    label='屏東縣公立犬貓中途之家',
                    text="功能建立中....QQ"
                )
            ),
            ImageCarouselColumn(
                image_url='http://dog.chiayi.gov.tw/',
                action=MessageAction(
                    label='嘉義市動物保護教育園區 ',
                    text="功能建立中....QQ"
                )
            )
        ]
    )
)

In [ ]:
#貓貓輪播圖文表單
unadopted_cat= TemplateSendMessage(
        alt_text='ImageCarouselTemplate',
        template=ImageCarouselTemplate(
        columns=[
            ImageCarouselColumn(
                image_url='https://scontent-tpe1-1.xx.fbcdn.net/v/t39.30808-6/358034680_6621665447855088_9072443692625844234_n.jpg?_nc_cat=111&ccb=1-7&_nc_sid=730e14&_nc_ohc=k5d4AzpNv4cAX8kuCX2&_nc_ht=scontent-tpe1-1.xx&oh=00_AfAA9kUXmnIjT00X2ACgkmaEPCaDqQ1ni57IeEbl017yTg&oe=64AFEAA2',
                action=MessageAction(
                    label='快帶我回家',
                    text="喵喵"
                )
            ),
            ImageCarouselColumn(
                image_url='https://scontent-tpe1-1.xx.fbcdn.net/v/t39.30808-6/358371209_6621665444521755_6325089610571750015_n.jpg?_nc_cat=102&ccb=1-7&_nc_sid=730e14&_nc_ohc=Tc1YhA38mFMAX9MCI0B&_nc_ht=scontent-tpe1-1.xx&oh=00_AfCvJwTY-9mtA7xBMnDq4FBH_WKxFJTzWTuF418guzjWjA&oe=64AF8008',
                action=MessageAction(
                    label='快帶我回家',
                    text="喵喵"
                )
            ),
            ImageCarouselColumn(
                image_url='https://scontent-tpe1-1.xx.fbcdn.net/v/t39.30808-6/358384637_6621665451188421_8571065255179003602_n.jpg?_nc_cat=110&ccb=1-7&_nc_sid=730e14&_nc_ohc=NVY4Ntw4AR8AX-WppZx&_nc_ht=scontent-tpe1-1.xx&oh=00_AfD0WQ1wJ8AaBhaV1l8RJIx0Po7oNHI_DQ2SjgWrEbMTqw&oe=64AEC7BF',
                action=MessageAction(
                    label='快帶我回家',
                    text="喵喵"
                )
            ),
            ImageCarouselColumn(
                image_url='https://scontent-tpe1-1.xx.fbcdn.net/v/t39.30808-6/358032200_6621665477855085_1432999987564834757_n.jpg?_nc_cat=111&ccb=1-7&_nc_sid=730e14&_nc_ohc=sOCGVEFE3-oAX8Pt-YO&_nc_ht=scontent-tpe1-1.xx&oh=00_AfDcVo5kcHzZovd_dR_zbiCKfvfkQ1DPD_kUmHDs5inePQ&oe=64AF3F3E',
                action=MessageAction(
                    label='快帶我回家',
                    text="喵喵"
                )
            )
        ]
    )
)

In [ ]:
'''

消息素材準備

'''

# 將消息模型，文字收取消息與文字寄發消息 引入

fill_doc= TextSendMessage(text="填寫申請文件\n https://forms.gle/xMiHhKcE2mktxUbN7")

coding_pending= TextSendMessage(text="功能建立中")
earn_some_money=TextSendMessage(text="來吧給你一些好貨")
diary_record=TextSendMessage("已將您的筆記內容記錄到筆記本")
question_solve=TextSendMessage(text='''您遇到甚麼問題呢?
               ''')
#心得記錄
diary_text=TextSendMessage("請輸入此次旅程的心得")
diary_image=TextSendMessage("請上傳這次最滿意的一張照片")

#自我身體健康評估(圖片)
analyze_healthy=ImageSendMessage(
    original_content_url='https://d1b8dyiuti31bx.cloudfront.net/NewsPhotos/20220421/50_031401430940.jpg',
    preview_image_url='https://d1b8dyiuti31bx.cloudfront.net/NewsPhotos/20220421/50_031401430940.jpg'
    )

#登山小知識(圖片)
hiking_tips=ImageSendMessage(
    original_content_url="https://54aming.com/uploads/elcitra/image/2482/42104891_458333264659993_7208547494697893888_o.jpg",
    preview_image_url="https://54aming.com/uploads/elcitra/image/2482/42104891_458333264659993_7208547494697893888_o.jpg"
    )

#裝備檢查資訊(圖片)
logging_check=ImageSendMessage(
    original_content_url="https://17jump.tw/wp-content/uploads/20200420201147_45.png",
    preview_image_url="https://17jump.tw/wp-content/uploads/20200420201147_45.png"
    )


'''

設計一個字典
當用戶輸入相應文字消息時，系統會從此挑揀消息

'''

#樣板3字典
template_message_dict3 = {
  "@start":main_template_message,
  # "@我想在山裡炊食": question_confirm,
  "@尋找水源!": coding_pending,
  "@營地搭建": coding_pending,
  "@體力補充":coding_pending
}


template_message_dict4={
    "我需要一些小知識":hiking_tips,
    "身體欠安需要調查一下":analyze_healthy,
    "我對我的記性感到不可靠":logging_check,
    "我對某些東西有點疑惑?":question_solve,
}



In [ ]:
# 主程序運行
port = 5000
# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))
app.run()

 * ngrok tunnel "https://8711-104-196-209-255.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://8711-104-196-209-255.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [26/Jul/2023 16:39:16] "POST / HTTP/1.1" 200 -


{"destination":"U02fd951784fe9dcc42fe1bf204efd4c5","events":[]}


...

INFO:werkzeug:127.0.0.1 - - [26/Jul/2023 16:40:16] "POST / HTTP/1.1" 200 -


{"destination":"U02fd951784fe9dcc42fe1bf204efd4c5","events":[{"type":"message","message":{"type":"image","id":"465665717254226050","contentProvider":{"type":"line"}},"webhookEventId":"01H69H7RA7QRGPMGJB9D5JTYXC","deliveryContext":{"isRedelivery":false},"timestamp":1690389700872,"source":{"type":"user","userId":"Ue1e7d62bf80e6386013ca1afc6bc147a"},"replyToken":"d90f613b76fa42bb8ebfec5506e8a2a0","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [26/Jul/2023 16:41:43] "POST / HTTP/1.1" 200 -


{"destination":"U02fd951784fe9dcc42fe1bf204efd4c5","events":[{"type":"message","message":{"type":"image","id":"465665829276221473","contentProvider":{"type":"line"}},"webhookEventId":"01H69H9SFWEFFTYXVBBYR6KAFX","deliveryContext":{"isRedelivery":false},"timestamp":1690389767564,"source":{"type":"user","userId":"Ue1e7d62bf80e6386013ca1afc6bc147a"},"replyToken":"4ed2642c6ae845aa857729bd08497cf6","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [26/Jul/2023 16:42:50] "POST / HTTP/1.1" 200 -
